<center><h1><font color='#000FFF'> Data Science Programming 2<br>
Bonus</font></h1><center>
<center><i>by Arailym Seisenbekkyzy<i><center>

# Introduction

In this assignment, I will write a neural network to classify whether images contain either a dog or a cat. Link to: https://www.kaggle.com/competitions/dogs-vs-cats/data

## Import all the packages that will need.

In [ ]:
# import packages
import os
import zipfile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import zipfile
import glob
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

## Load your data and explore it.

In [ ]:
# load  dataset

zip_file = glob.glob('/kaggle/input/dogs-vs-cats/*.zip')  #return any files with .zip extension
print(zip_file)

#extract file into a temp folder
def extract_zip(file):
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall("temp")
        
#extract both train and test1 zip
for files in zip_file:
    extract_zip(files)

print(len(os.listdir('/kaggle/working/temp/train')), "training data")
print(len(os.listdir('/kaggle/working/temp/test1')), "test data")
os.listdir("temp")

Since in the future I will be converting our data to a **Dataframe**, we will create a target variable responsible for the picture class: **cat** and **dog**

In [ ]:
# explore

'''
create a target variable responsible for the picture class:
cat and dog, because then I convert data to a Dataframe
'''

path = '/kaggle/working/temp/train/'
filenames = os.listdir(path)
filenames[:5]

label = []
for filename in filenames:
    if filename.split('.')[0] =='cat': # cat = 1
        label.append('1')
    else:
        label.append('0') # dog = 0
        
df = pd.DataFrame({
                   'name':filenames,
                   'label':label
                 })
df.head()

In [ ]:
#histogram to check whether the label is correct sorted all the photos into classes

print(df['label'].value_counts())
sns.countplot(data=df, x=df['label']);

We have an equal number of cat and dog classes
* cat = 1
*dog = 0

Let's extract a couple of images from the data:

In [ ]:
load_img(path+'cat.10001.jpg')

## Describe chosen model and its architecture: Base model

#### Basic architecture of **CNN**

Convolutional Neural Network (CNN) has been the to-go neural network architecture when dealing with images. From Yann LeCunn's LeNet in 1990s predicting digits on the popular MNIST data, to Alexnet impressive performance on ImageNet, the area of CNN has evolved tremendously.


In this section, we will try to implement a CNN architecture from scratch, using the Keras library. We will be using just 5 epochs for all our training

In this model, we use 5 consecutive blocks from ***Conv2D*** and ***MaxPooling2d*** with different filtering depths.
Since we have a classification task, after 5 blocks we will transform our data into a 1D tensor and apply Dense Layers.
The last ***dense layer*** should have 1 layer and ***activation="sigmoid"*** since we have a binary classification task
input_shape set ***(256, 256, 3)*** as a base for further use of the generator without resizing the image

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
#optimizer='Adam' as the most basic and recommended
#loss='binary_crossentropy' since we have a binary classification
#metrics='acc' since we have the same number of classes and accuracy is a suitable metric

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics='acc')

## Describe how you will divide data for training, validation, and testing.

Splitting data into train, test, valid
We will use proportions train:test:valid - ***8:1:1***
- On the ***train*** sample, we will train our model
- On the ***validation*** sample, we will check the ability of our model to generalize to unknown data
- On the ***test*** sample, we will make the final prediction

In [ ]:
# split the data in a ratio of 8:1:1 and with the same ratio of classes

train, test_val = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=17)
test, val = train_test_split(test_val, test_size=0.5,  stratify=test_val['label'], random_state=17)

print('train size:', train.shape[0],
      '\nvalidation size:', val.shape[0],
      '\ntest size:', test.shape[0],     
     )
print('train labels:\n',train['label'].value_counts(),
      '\n\nvalidataion labels:\n',val['label'].value_counts(),
      '\n\ntest labels:\n',test['label'].value_counts(),
      sep='')

# Describe your evaluation metrics.

Apply an ImageDataGenerator to our data to make it look like an input_shape for our model.

Neural networks need to receive scaled data as input, for this we apply rescale=1./255

In this case, the image size is not specified, because flow_from_dataframe creates target_size=(256, 256).

Augmentation is the process of generating artificial images using rotations, mirroring, shifts and many other different methods based on existing ones.

In [ ]:
# use and apply augmentation

# batch_size=32
# train_size(20000)/batch_size(32)=625
# larger the batch_size value, the less time it takes to train one epoch and the more epochs are needed to get good results
train_gen = ImageDataGenerator(rescale=1./255)
train_data = train_gen.flow_from_dataframe(train,
                                           directory=path,
                                           x_col='name',
                                           y_col='label',
                                           class_mode='binary',
                                           seed=17                                          
                                          )

val_gen = ImageDataGenerator(rescale=1./255)
val_data = val_gen.flow_from_dataframe(val,
                                       directory=path,
                                       x_col='name',
                                       y_col='label',
                                       class_mode='binary',
                                       seed=17  
                                      )

## Train model.

In [ ]:
history = model.fit(train_data,
                    validation_data = val_data,
                    epochs=30
                   )

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(15,8))
plt.plot(loss, label='Train loss')
plt.plot(val_loss,'--', label='Val loss')
plt.title('Training and validation loss')
plt.xticks(np.arange(0,10))
plt.yticks(np.arange(0, 0.7, 0.05));
plt.grid()
plt.legend();



We see the problem of overfitting, since after the 5th epoch, val loss stopped decreasing, but went up.

# Data Preprocessing : Augmentation

Augmentation is the process of generating artificial images using rotations, mirroring, shifts and other methods based on existing data.

In [ ]:
# Data Preprocessing : Augmentation
aug_gen = ImageDataGenerator(rescale = 1./255,
                               shear_range = 0.2,
                               zoom_range = 0.2,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest'
                              )

img_cat = load_img(path+'cat.10001.jpg')
img_dog = load_img(path+'dog.1280.jpg')

img_cat_arr = image.img_to_array(img_cat)
img_cat_arr = img_cat_arr.reshape((1,)+img_cat_arr.shape)

img_dog_arr = image.img_to_array(img_dog)
img_dog_arr = img_dog_arr.reshape((1,)+ img_dog_arr.shape)

aug_images_cat = aug_gen.flow(img_cat_arr, batch_size=1)
aug_images_dog = aug_gen.flow(img_dog_arr, batch_size=1)

plt.figure(figsize=(15,8))
plt.subplot(141)
plt.imshow(img_cat)
plt.title("original")
i=2
for batch in aug_images_cat:
    plt.subplot(14*10+i)
    plt.imshow(image.array_to_img(batch[0]))
    plt.title("augmented")
    i += 1
    if i % 5 == 0:
        break

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(141)
plt.imshow(img_dog)
plt.title("original")
i=2
for batch in aug_images_dog:
    plt.subplot(14*10+i)
    plt.imshow(image.array_to_img(batch[0]))
    plt.title("augmented")
    i += 1
    if i % 5 == 0:
        break

Apply augmentation to our train data.
It is important to apply augmentation only to train data, not to val and test. We need to provide our model with a large sample of images so that it can learn to recognize patterns in images.

In [ ]:
train_data = aug_gen.flow_from_dataframe(train,
                                         directory=path,
                                         x_col='name',
                                         y_col='label',
                                         class_mode='binary',
                                         target_size=(224,224),
                                         seed=17
                                        )

val_data = val_gen.flow_from_dataframe(val,
                                       directory=path,
                                       x_col='name',
                                       y_col='label',
                                       class_mode='binary',
                                       target_size=(224,224),
                                       seed=17  
                                      )

## Evaluate model.

**Base model:**

In [ ]:
# howbase model looked
model.summary()

It is important to understand that a strong fight against an overfitting problem can turn into an underfitting problem.

Perhaps the augmentation is already enough to solve the problem of overfitting our model, but I want to add some more examples of hyperparameter regularization

In [ ]:
best_model = models.Sequential()

best_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
best_model.add(layers.MaxPooling2D((2, 2)))

best_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
best_model.add(layers.MaxPooling2D((2, 2)))

best_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
best_model.add(layers.MaxPooling2D((2, 2)))

best_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
best_model.add(layers.MaxPooling2D((2, 2)))

best_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
best_model.add(layers.MaxPooling2D((2, 2)))

best_model.add(layers.Flatten())
best_model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
best_model.add(layers.Dropout(0.2))
best_model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
best_model.summary()

Try to directly set the learning_rate to the optimizer, take a value slightly less than the default.

Thus, we guarantee that our optimizer will not get stuck in the local minimum of the function, however, for this we reduce the learning rate

In [ ]:
best_model.compile(optimizer=optimizers.Adam(learning_rate=5e-4), loss='binary_crossentropy', metrics='acc')

# Tuned model training

Increase the number of **epochs to 60**, most likely the model will need many more epochs than in the previous version

Also add **callbacks** that will stop training if **val_loss** has not changed more than **0.001** over the past **5 epochs**

In [ ]:
# increase the number of epochs to 60

history_2 = best_model.fit(train_data,
                           validation_data = val_data,
                           epochs=60,
                           callbacks=[EarlyStopping(monitor='val_acc', min_delta=0.001, patience=5, verbose=1)]
                          )

Our network stopped after 19 epochs as the callback fired.

Perhaps after a few epochs, val_acc would begin to grow, but this accuracy suits me, since this notebook was not written to find the best results, but to work out and understand working with CNN in Keras.


In [ ]:
# accuracy plot
loss = history_2.history['acc']
val_loss = history_2.history['val_acc']

plt.figure(figsize=(15,8))
plt.plot(loss, label='Train acc')
plt.plot(val_loss,'--', label='Val acc')
plt.title('Training and validation accuracy')
plt.yticks(np.arange(0.5, 1, 0.05))
plt.xticks(np.arange(0, 26))
plt.grid()
plt.legend();

Compared to our first model, we really got rid of the **overfitting** problem. Now our metrics on the chart grow in proportion to each other

In [ ]:
# save the model and apply it to the test data
best_model.save('best_model_cat_vs_dog.h5')

# Result on test data

In [ ]:
test_data = val_gen.flow_from_dataframe(test,
                                        directory=path,
                                        x_col='name',
                                        y_col='label',
                                        class_mode='binary',
                                        target_size=(224,224),
                                        shuffle=False,
                                        seed=17  
                                       )
test_pred = best_model.predict(test_data)
pred_label = test_pred > 0.5
true_label = test_data.classes
ConfusionMatrixDisplay(confusion_matrix(true_label, pred_label), display_labels=test_data.class_indices).plot();

In [ ]:
best_model.evaluate(test_data)

**92% on test data**

It is possible that the model could be trained for high accuracy results, however, the purpose of this notebook was to learn how to work with **Keras** and **CNN**